In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
from typing import Tuple

class Flatten(nn.Module):
    """A custom layer that views an input as 1D."""
    
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class MultivariateCNN(nn.Module):

    def __init__(self, 
                 input_dimension: Tuple,
                 in_channels: int,
                 n_outputs: int,
                 n_cnn_layers: int = 2, 
                 conv_kernel_size: int=2, 
                 pool_kernel_size: int=2):
        super(MultivariateCNN, self).__init__()
        self.linear_dim1 = get_cnn_layer_output_dim(n_layers=n_cnn_layers, 
                                                    input_size=input_dimension[0], 
                                                    conv_kernel_size=conv_kernel_size)
        self.linear_dim2 = get_cnn_layer_output_dim(n_layers=n_cnn_layers, 
                                                    input_size=input_dimension[1], 
                                                    conv_kernel_size=conv_kernel_size)
        self.encoder = nn.Sequential(
              nn.Conv2d(in_channels, out_channels=8, kernel_size=conv_kernel_size),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=pool_kernel_size),
              nn.Conv2d(in_channels=8, out_channels=16, kernel_size=conv_kernel_size),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=pool_kernel_size),
              Flatten(),
              nn.Linear(self.linear_dim1*self.linear_dim2*16, 128),
              nn.Dropout(0.5),
        )
        self.linear_classifiers = [nn.Linear(128, n_outputs) for i in range(in_channels)]

    def forward(self, x):
        out = self.encoder(x)
        outputs = [classifier(out) for classifier in self.linear_classifiers]
        return outputs

In [ ]:
N_CLASSES = 5
N_SERIES = 3
N_DIM1 = 24
N_DIM2 = 43

model = MultivariateCNN(input_dimension=(N_DIM1, N_DIM2), in_channels=N_SERIES, n_outputs=N_CLASSES, 
                        n_cnn_layers=2, conv_kernel_size=5, pool_kernel_size=2
                       ).to(device)
print(model)


In [ ]:
X = torch.rand(1, N_SERIES, N_DIM1, N_DIM2, device=device)
logits = model(X)
pred_probabs = [nn.Softmax(dim=1)(logit) for logit in logits]
y_preds = [pred_probab.argmax(1) for pred_probab in pred_probabs]
print(f"Predicted class: {y_preds}")

In [ ]:
X

In [ ]:
nn.Softmax(dim=1)(logits[0])

In [ ]:
input = torch.randn(20, 16, 50, 100)
m = nn.Conv2d(16, 33, 3, stride=1)
output = m(input)
output.shape

In [ ]:
3136/128

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())


In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())


In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")
